<a href="https://colab.research.google.com/github/Shalululuz/dataviz2025/blob/main/student_health_warehouse_lab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📊 Lab: สร้าง Health Data Warehouse (ฉบับนักศึกษา)

**ชื่อ**: ___________________________  **รหัสนักศึกษา**: _______________

## 🎯 วัตถุประสงค์:
ฝึกปฏิบัติการสร้าง Data Warehouse จากฐานข้อมูลระบบเฝ้าระวังโรค NCDs

## 📚 สิ่งที่จะได้เรียนรู้:
1. การเชื่อมต่อและดึงข้อมูลจาก SQLite Database
2. การทำ Data Transformation (JOIN, Groupby, Merge)
3. การสร้าง Data Warehouse 5 ตาราง
4. การอัปโหลดข้อมูลไปยัง Google Sheets

## ⏱️ เวลาที่ใช้: 120 นาที

---

## 📦 Part 1: เตรียมสภาพแวดล้อม (10 นาที)

### Exercise 1.1: ติดตั้ง Libraries

**โจทย์**: ติดตั้ง packages ที่จำเป็นสำหรับ Lab นี้

**Hint**: ต้องติดตั้ง `gspread`, `google-auth`, `google-auth-oauthlib`, `google-auth-httplib2`, `openpyxl`

In [1]:
# TODO: ติดตั้ง packages ที่จำเป็น
# Hint: ใช้ !pip install ... -q

# เขียนโค้ดที่นี่


print("✅ ติดตั้ง libraries สำเร็จ")

✅ ติดตั้ง libraries สำเร็จ


### Exercise 1.2: Mount Google Drive

**โจทย์**: Mount Google Drive และตรวจสอบว่ามีไฟล์ที่ต้องการหรือไม่

**ไฟล์ที่ต้องการ**:
- `ncds_sample.db`
- `districts_with_zone_20250930_084409.xlsx`

**Path**: `/content/drive/MyDrive/Dashboard-Teaching-2025/`

In [8]:
# TODO: Mount Google Drive
# Hint: ใช้ from google.colab import drive
#       drive.mount('/content/drive')

# เขียนโค้ดที่นี่
from google.colab import drive
from os.path import exists
drive.mount('/content/drive')


# TODO: กำหนด path ของไฟล์
import os

SHARED_FOLDER = '/content/drive/MyDrive/Dataviz2025/health_warehouse_tutorial/student_health_warehouse_lab.ipynb'
DB_PATH = '/content/drive/MyDrive/Dataviz2025/health_warehouse_tutorial/ncds_sample.db'
DISTRICTS_PATH = '/content/drive/MyDrive/Dataviz2025/health_warehouse_tutorial/districts_with_zone_20250930_084409.xlsx'

# TODO: ตรวจสอบว่าไฟล์มีอยู่จริง
# Hint: ใช้ os.path.exists()

# เขียนโค้ดที่นี่
if exists(DB_PATH):
    print(f"✅ Found database file: {DB_PATH}")
else:
    print(f"❌ Database file not found: {DB_PATH}")

if exists(DISTRICTS_PATH):
    print(f"✅ Found districts file: {DISTRICTS_PATH}")
else:
    print(f"❌ Districts file not found: {DISTRICTS_PATH}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Found database file: /content/drive/MyDrive/Dataviz2025/health_warehouse_tutorial/ncds_sample.db
✅ Found districts file: /content/drive/MyDrive/Dataviz2025/health_warehouse_tutorial/districts_with_zone_20250930_084409.xlsx


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### Exercise 1.3: Import Libraries

**โจทย์**: Import libraries ที่จำเป็นทั้งหมด

In [9]:
# TODO: Import libraries ที่จำเป็น
# Hint: sqlite3, pandas, numpy, datetime, gspread, warnings

# เขียนโค้ดที่นี่
import sqlite3
import pandas as pd
import numpy as np
import datetime
import gspread
import warnings


warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

print("✅ Import libraries สำเร็จ")

✅ Import libraries สำเร็จ


---

## 🔌 Part 2: เชื่อมต่อและดึงข้อมูล (15 นาที)

### Exercise 2.1: เชื่อมต่อ SQLite Database

**โจทย์**: เชื่อมต่อกับ `ncds_sample.db` และแสดงรายชื่อตารางทั้งหมดพร้อมจำนวนแถว

In [10]:
# TODO: เชื่อมต่อ SQLite Database
# Hint: ใช้ sqlite3.connect(DB_PATH)

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# TODO: ดูรายชื่อตารางทั้งหมดพร้อมจำนวนแถว
# Hint: SELECT name FROM sqlite_master WHERE type='table'
#       จากนั้น SELECT COUNT(*) FROM {table_name}

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
tables = cursor.fetchall()

for table in tables:
    table_name = table[0]
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    count = cursor.fetchone()[0]
    print(f"Table: {table_name}, Rows: {count}")

Table: campaign, Rows: 20
Table: ncds_profile, Rows: 192
Table: ncds_record, Rows: 192
Table: ncds_habit, Rows: 192
Table: memberdata, Rows: 1920


### Exercise 2.2: ดึงข้อมูลจากตาราง

**โจทย์**: ดึงข้อมูลจาก 5 ตารางหลัก:
1. `ncds_profile`
2. `ncds_record`
3. `ncds_habit`
4. `campaign`
5. `memberdata`

**Hint**: ใช้ `pd.read_sql_query("SELECT * FROM table_name", conn)`

In [12]:
# TODO: ดึงข้อมูลจากแต่ละตาราง

print("📥 กำลังดึงข้อมูลจาก Database...")

# 1. ncds_profile
ncds_profile = pd.read_sql_query("SELECT * FROM ncds_profile", conn)
print(f"✅ ncds_profile: {len(ncds_profile)} แถว")

# 2. ncds_record
ncds_record = pd.read_sql_query("SELECT * FROM ncds_record", conn)
print(f"✅ ncds_record: {len(ncds_record)} แถว")

# 3. ncds_habit
ncds_habit = pd.read_sql_query("SELECT * FROM ncds_habit", conn)
print(f"✅ ncds_habit: {len(ncds_habit)} แถว")

# 4. campaign
campaign = pd.read_sql_query("SELECT * FROM campaign", conn)
print(f"✅ campaign: {len(campaign)} แถว")

# 5. memberdata
memberdata = pd.read_sql_query("SELECT * FROM memberdata", conn)
print(f"✅ memberdata: {len(memberdata)} แถว")

📥 กำลังดึงข้อมูลจาก Database...
✅ ncds_profile: 192 แถว
✅ ncds_record: 192 แถว
✅ ncds_habit: 192 แถว
✅ campaign: 20 แถว
✅ memberdata: 1920 แถว


### Exercise 2.3: โหลดข้อมูลพื้นที่

**โจทย์**: โหลดข้อมูลพื้นที่จากไฟล์ Excel และแปลง `district_code` เป็น string

In [13]:
# TODO: โหลดข้อมูลพื้นที่
# Hint: ใช้ pd.read_excel()

districts_df = pd.read_excel(DISTRICTS_PATH)
districts_df['district_code'] = districts_df['district_code'].astype(str)

print(f"✅ โหลดข้อมูลพื้นที่: {len(districts_df)} แถว")
display(districts_df.head())

✅ โหลดข้อมูลพื้นที่: 7356 แถว


,district_code,district_name,amphur_code,amphur_name,province_code,province_name,lat,lng,zone
0,100101,แขวง พระบรมมหาราชวัง,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.492,4.0
1,100102,แขวง วังบูรพาภิรมย์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.744,100.499,4.0
2,100103,แขวง วัดราชบพิธ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.750,100.499,4.0
3,100104,แขวง สำราญราษฎร์,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.751,100.503,4.0
4,100105,แขวง ศาลเจ้าพ่อเสือ,1001,เขต พระนคร,10,กรุงเทพมหานคร,13.754,100.497,4.0


---

## 🏗️ Part 3: สร้าง Data Warehouse (40 นาที)

### Exercise 3.1: Health Profile Warehouse

**โจทย์**: สร้างตาราง Health Profile Warehouse ตามขั้นตอนต่อไปนี้:

**ขั้นตอน**:
1. ดึงข้อมูล**ล่าสุด**ของแต่ละสมาชิกจาก `ncds_profile` และ `ncds_record`
2. นับจำนวนครั้งการตรวจของแต่ละสมาชิก
3. JOIN กับ `campaign` เพื่อได้ `district_code`
4. JOIN กับ `districts_df` เพื่อได้ข้อมูลพื้นที่
5. สร้าง DataFrame ตามคอลัมน์ที่กำหนด

**คอลัมน์ที่ต้องมี**:
- member_id
- จำนวนครั้งการตรวจ
- เพศ (แปลง 0→หญิง, 1→ชาย)
- อายุ
- ตำบล_id, ตำบล
- อำเภอ_id, อำเภอ
- จังหวัด_id, จังหวัด
- เขต
- กลุ่มอายุ
- ประวัติการเจ็บป่วย (code และ label)
- ประวัติแพ้ยา (code และ label)
- ประวัติแพ้อาหาร (code และ label)
- ญาติสายตรงNCDs (code และ label)

In [17]:
# Step 1: ดึงข้อมูลล่าสุดของแต่ละสมาชิก
# Hint: ใช้ .sort_values('dateadd').groupby('memberid').tail(1)

latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

print(f"✓ Latest profiles: {len(latest_profiles)} แถว")
print(f"✓ Latest records: {len(latest_records)} แถว")

✓ Latest profiles: 100 แถว
✓ Latest records: 100 แถว


In [19]:
# Step 2: นับจำนวนครั้งการตรวจ
# Hint: ใช้ .groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

screening_count = # เขียนโค้ดที่นี่

print(f"✓ Screening count: {len(screening_count)} สมาชิก")
screening_count.head()

✓ Screening count: 100 สมาชิก


,memberid,จำนวนครั้งการตรวจ
0,10000000,3
1,10000001,3
2,10000002,3
3,10000003,2
4,10000004,2


In [20]:
# Step 3: JOIN กับ campaign
# Hint: ใช้ pd.merge() โดย left_on='campaignId', right_on='id'

campaign['district_code'] = campaign['district_code'].astype(str)

profile_campaign = # เขียนโค้ดที่นี่

print(f"✓ Profile + Campaign: {len(profile_campaign)} แถว")

SyntaxError: invalid syntax (ipython-input-908266053.py, line 6)

In [ ]:
# Step 4: JOIN กับ districts
# Hint: ใช้ pd.merge() โดย on='district_code'

profile_location = # เขียนโค้ดที่นี่

print(f"✓ Profile + Location: {len(profile_location)} แถว")

In [ ]:
# Step 5: JOIN กับ record และ screening_count
# Hint: JOIN 2 ครั้ง - ครั้งแรกกับ latest_records, ครั้งที่สองกับ screening_count

profile_record = # เขียนโค้ดที่นี่ (JOIN กับ latest_records)

final_data = # เขียนโค้ดที่นี่ (JOIN กับ screening_count)

# แปลง NaN ในจำนวนครั้งการตรวจเป็น 0
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

print(f"✓ Final data: {len(final_data)} แถว")
final_data.head()

In [ ]:
# Step 6: สร้าง Helper Functions
# TODO: เขียนฟังก์ชันแปลงค่าต่างๆ

def get_gender_label(gender):
    """แปลงเพศ: 0→หญิง, 1→ชาย, อื่นๆ→999"""
    # เขียนโค้ดที่นี่
    pass

def get_age_group(age):
    """
    แปลงกลุ่มอายุ:
    - < 20: 'อายุต่ำกว่า 20 ปี'
    - 20-34: 'อายุระหว่าง 20-34 ปี'
    - 35-59: 'อายุระหว่าง 35-59 ปี'
    - >= 60: 'อายุ 60 ปีขึ้นไป'
    """
    # เขียนโค้ดที่นี่
    pass

def convert_illness_history(value):
    """
    แปลงประวัติการเจ็บป่วย:
    - -1: 'ไม่มีประวัติ'
    - 1: 'มีประวัติการเจ็บป่วยรวม'
    - อื่นๆ: 'ไม่ทราบ'
    """
    # เขียนโค้ดที่นี่
    pass

# TODO: เขียนฟังก์ชันเพิ่มเติมสำหรับ drug_allergy, food_allergy, relatives_ncds
# (รูปแบบเหมือน convert_illness_history)


In [ ]:
# Step 7: สร้าง Health Profile Warehouse
# TODO: สร้าง DataFrame ตามคอลัมน์ที่กำหนด

health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': # เขียนโค้ดที่นี่ (ใช้ get_gender_label)
    'อายุ': # เขียนโค้ดที่นี่
    # ... เพิ่มคอลัมน์อื่นๆ
})

print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
health_profile_wh.head()

**NCDs Disease Warehouse**

In [22]:
# ดึงข้อมูลล่าสุดของแต่ละสมาชิก
latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)
screening_count = ncds_record.groupby('memberid').size().reset_index(name='จำนวนครั้งการตรวจ')

# JOIN ข้อมูล
campaign['district_code'] = campaign['district_code'].astype(str)
profile_campaign = pd.merge(latest_profiles, campaign[['id', 'district_code']],
                            left_on='campaignId', right_on='id', how='left')
profile_location = pd.merge(profile_campaign, districts_df, on='district_code', how='left')
profile_record = pd.merge(profile_location,
                         latest_records[['memberid', 'drug_allergy', 'food_allergy', 'record_ncds', 'relatives_ncds']],
                         on='memberid', how='left')
final_data = pd.merge(profile_record, screening_count, on='memberid', how='left')
final_data['จำนวนครั้งการตรวจ'] = final_data['จำนวนครั้งการตรวจ'].fillna(0).astype(int)

# Helper functions
def handle_text_data(value):
    return 999 if pd.isna(value) or value == '' else str(value)

def handle_numeric_data(value):
    return 999 if pd.isna(value) else float(value)

def get_gender_label(gender):
    if pd.isna(gender): return 999
    return "หญิง" if int(gender) == 0 else "ชาย"

def get_age_group(age):
    if pd.isna(age) or age == 999 or age == 0: return 999
    if age < 20: return 'อายุต่ำกว่า 20 ปี'
    if age < 35: return 'อายุระหว่าง 20-34 ปี'
    if age < 60: return 'อายุระหว่าง 35-59 ปี'
    return 'อายุ 60 ปีขึ้นไป'

def convert_value(value, labels):
    """แปลงค่า -1, 1 เป็น label"""
    if pd.isna(value) or value == '' or value == 999: return labels[0]
    if value == -1: return labels[1]
    if value == 1: return labels[2]
    return labels[0]

# สร้าง Warehouse
health_profile_wh = pd.DataFrame({
    'member_id': final_data['memberid'],
    'จำนวนครั้งการตรวจ': final_data['จำนวนครั้งการตรวจ'],
    'เพศ': final_data['gender'].apply(get_gender_label),
    'อายุ': final_data['age'].apply(handle_numeric_data),
    'ตำบล_id': final_data['district_code'].apply(handle_text_data),
    'ตำบล': final_data['district_name'].apply(handle_text_data),
    'อำเภอ_id': final_data['amphur_code'].apply(handle_numeric_data),
    'อำเภอ': final_data['amphur_name'].apply(handle_text_data),
    'จังหวัด_id': final_data['province_code'].apply(handle_numeric_data),
    'จังหวัด': final_data['province_name'].apply(handle_text_data),
    'เขต': final_data['zone'].apply(handle_numeric_data),
    'กลุ่มอายุ': final_data['age'].apply(get_age_group),
    'ประวัติการเจ็บป่วย_code': final_data['record_ncds'].apply(handle_numeric_data),
    'ประวัติการเจ็บป่วย': final_data['record_ncds'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม'])),
    'ประวัติแพ้ยา_code': final_data['drug_allergy'].apply(handle_numeric_data),
    'ประวัติแพ้ยา': final_data['drug_allergy'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้ยา'])),
    'ประวัติแพ้อาหาร_code': final_data['food_allergy'].apply(handle_numeric_data),
    'ประวัติแพ้อาหาร': final_data['food_allergy'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการแพ้อาหาร'])),
    'ญาติสายตรงNCDs_code': final_data['relatives_ncds'].apply(handle_numeric_data),
    'ญาติสายตรงNCDs': final_data['relatives_ncds'].apply(lambda x: convert_value(x, ['ไม่ทราบ', 'ไม่มีประวัติ', 'มีประวัติการเจ็บป่วยรวม']))
})

print(f"✅ Health Profile Warehouse: {len(health_profile_wh)} แถว")
health_profile_wh.head()

✅ Health Profile Warehouse: 100 แถว


,member_id,จำนวนครั้งการตรวจ,เพศ,อายุ,ตำบล_id,ตำบล,อำเภอ_id,อำเภอ,จังหวัด_id,จังหวัด,เขต,กลุ่มอายุ,ประวัติการเจ็บป่วย_code,ประวัติการเจ็บป่วย,ประวัติแพ้ยา_code,ประวัติแพ้ยา,ประวัติแพ้อาหาร_code,ประวัติแพ้อาหาร,ญาติสายตรงNCDs_code,ญาติสายตรงNCDs
0,10000036,1,ชาย,50.0,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
1,10000058,2,หญิง,44.0,120102,ต. ตลาดขวัญ,1201.0,อ. เมืองนนทบุรี,12.0,จ. นนทบุรี,6.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
2,10000025,1,หญิง,73.0,400101,ต. ในเมือง,4001.0,อ. เมืองขอนแก่น,40.0,จ. ขอนแก่น,1.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,1.0,มีประวัติการแพ้ยา,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ
3,10000040,1,หญิง,48.0,500101,ต. ศรีภูมิ,5001.0,อ. เมืองเชียงใหม่,50.0,จ. เชียงใหม่,2.0,อายุระหว่าง 35-59 ปี,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,1.0,มีประวัติการเจ็บป่วยรวม
4,10000076,1,หญิง,62.0,100101,แขวง พระบรมมหาราชวัง,1001.0,เขต พระนคร,10.0,กรุงเทพมหานคร,4.0,อายุ 60 ปีขึ้นไป,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ,-1.0,ไม่มีประวัติ


### Exercise 3.2: NCDs Disease Warehouse

**โจทย์**: สร้างตารางประวัติโรค NCDs

**ขั้นตอน**:
1. ดึงข้อมูลล่าสุดจาก `ncds_record`
2. สำหรับแต่ละสมาชิก สร้างรายการโรคของผู้ตรวจ (8 โรค)
3. สร้างรายการโรคของญาติ (9 โรค)
4. รวมเป็นตารางเดียว โดยแต่ละแถว = 1 โรค

**โรคของผู้ตรวจ**: cancer, chronic_kidney_disease, cirrhosis, dementia, diabetes, hyperlipidemia, hypertension, stroke

**โรคของญาติ**: cancer, cirrhosis, coronary_artery_disease, dementia, diabetes, hyperlipidemia, hypertension, stroke, obesity

In [25]:
# TODO: สร้าง NCDs Disease Warehouse
# Hint: ดูตัวอย่างจากไฟล์ detailed_health_warehouse_tutorial.ipynb

# เขียนโค้ดที่นี่

def create_ncds_disease_warehouse(ncds_record):
    """
    สร้างตาราง NCDs Disease Warehouse

    Returns:
        DataFrame: NCDs Disease Warehouse
    """
    print("🏗️ สร้าง NCDs Disease Warehouse...")

    # ดึงข้อมูลล่าสุด
    latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)

    # Mapping โรค
    personal_disease_mapping = {
        'record_cancer': 'โรคมะเร็ง',
        'record_chronic_kidney_disease': 'โรคไตเรื้อรัง',
        'record_cirrhosis': 'โรคตับแข็ง',
        'record_dementia': 'โรคสมองเสื่อม',
        'record_diabetes': 'โรคเบาหวาน',
        'record_hyperlipidemia': 'โรคไขมันในเลือด',
        'record_hypertension': 'โรคความดันโลหิตสูง',
        'record_stroke': 'โรคหลอดเลือดสมอง'
    }

    relatives_disease_mapping = {
        'relatives_cancer': 'โรคมะเร็ง',
        'relatives_cirrhosis': 'โรคตับแข็ง',
        'relatives_coronary_artery_disease': 'โรคหลอดเลือดหัวใจ',
        'relatives_dementia': 'โรคสมองเสื่อม',
        'relatives_diabetes': 'โรคเบาหวาน',
        'relatives_hyperlipidemia': 'โรคไขมันในเลือด',
        'relatives_hypertension': 'โรคความดันโลหิตสูง',
        'relatives_stroke': 'โรคหลอดเลือดสมอง',
        'relatives_obesity': 'โรคอ้วนลงพุง'
    }

    combined_disease_records = []

    for _, row in latest_records.iterrows():
        member_id = row['memberid']

        # รายการโรคของผู้ตรวจ
        personal_diseases = []
        for disease_code, disease_name in personal_disease_mapping.items():
            if disease_code in row and pd.notna(row[disease_code]) and row[disease_code] == 1:
                personal_diseases.append({'disease_code': disease_code, 'disease_name': disease_name})

        # รายการโรคของญาติ
        relatives_diseases = []
        for disease_code, disease_name in relatives_disease_mapping.items():
            if disease_code in row and pd.notna(row[disease_code]) and row[disease_code] == 1:
                relatives_diseases.append({'disease_code': disease_code, 'disease_name': disease_name})

        max_diseases = max(len(personal_diseases), len(relatives_diseases))

        if max_diseases == 0:
            # ไม่มีโรคเลย
            combined_disease_records.append({
                'member_id': member_id,
                'no': 999,
                'โรค_code': 999,
                'โรคของผู้ตรวจ': 999,
                'no_โรคญาติผู้ตรวจ': 999,
                'โรคญาติผู้ตรวจ_code': 999,
                'โรคของญาติผู้ตรวจ': 999
            })
        else:
            # มีโรค - สร้างแถวตามจำนวนโรค
            for i in range(max_diseases):
                personal_no = i + 1 if i < len(personal_diseases) else 999
                personal_code = personal_diseases[i]['disease_code'] if i < len(personal_diseases) else 999
                personal_name = personal_diseases[i]['disease_name'] if i < len(personal_diseases) else 999

                relatives_no = i + 1 if i < len(relatives_diseases) else 999
                relatives_code = relatives_diseases[i]['disease_code'] if i < len(relatives_diseases) else 999
                relatives_name = relatives_diseases[i]['disease_name'] if i < len(relatives_diseases) else 999

                combined_disease_records.append({
                    'member_id': member_id,
                    'no': personal_no,
                    'โรค_code': personal_code,
                    'โรคของผู้ตรวจ': personal_name,
                    'no_โรคญาติผู้ตรวจ': relatives_no,
                    'โรคญาติผู้ตรวจ_code': relatives_code,
                    'โรคของญาติผู้ตรวจ': relatives_name
                })

    combined_disease_warehouse = pd.DataFrame(combined_disease_records)
    print(f"✅ สร้าง NCDs Disease Warehouse: {len(combined_disease_warehouse)} แถว")
    return combined_disease_warehouse

# สร้าง Warehouse
ncds_disease_wh = create_ncds_disease_warehouse(ncds_record)

# แสดงผลลัพธ์
print("\n📋 ตัวอย่างข้อมูล NCDs Disease Warehouse:")
ncds_disease_wh.head(10)



🏗️ สร้าง NCDs Disease Warehouse...
✅ สร้าง NCDs Disease Warehouse: 308 แถว

📋 ตัวอย่างข้อมูล NCDs Disease Warehouse:


,member_id,no,โรค_code,โรคของผู้ตรวจ,no_โรคญาติผู้ตรวจ,โรคญาติผู้ตรวจ_code,โรคของญาติผู้ตรวจ
0,10000036,999,999,999,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
1,10000036,999,999,999,2,relatives_dementia,โรคสมองเสื่อม
2,10000036,999,999,999,3,relatives_stroke,โรคหลอดเลือดสมอง
3,10000058,1,record_cancer,โรคมะเร็ง,1,relatives_diabetes,โรคเบาหวาน
4,10000058,2,record_dementia,โรคสมองเสื่อม,999,999,999
5,10000058,3,record_hypertension,โรคความดันโลหิตสูง,999,999,999
6,10000058,4,record_stroke,โรคหลอดเลือดสมอง,999,999,999
7,10000025,1,record_cirrhosis,โรคตับแข็ง,1,relatives_coronary_artery_disease,โรคหลอดเลือดหัวใจ
8,10000025,2,record_hypertension,โรคความดันโลหิตสูง,2,relatives_dementia,โรคสมองเสื่อม
9,10000025,999,999,999,3,relatives_obesity,โรคอ้วนลงพุง


### Exercise 3.3: Eating Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมการกิน

**ข้อมูลที่ต้องแปลง**:
- กินผัก (vegetable): 0→ไม่ทำเลย, 1→1-3 วัน, 2→4-6 วัน, 3→ทุกวัน
- กินขนมหวาน (dessert): 0→ทุกวัน, 1→4-6 วัน, 2→1-3 วัน, 3→ไม่ทำเลย (กลับด้าน!)
- ฯลฯ

In [24]:
# TODO: สร้าง Eating Behavior Warehouse
# Hint: ต้องสร้างฟังก์ชันแปลงค่าสำหรับแต่ละพฤติกรรม

# เขียนโค้ดที่นี่
def create_eating_behavior_warehouse(ncds_habit):
    """
    สร้างตาราง Eating Behavior Warehouse

    Returns:
        DataFrame: Eating Behavior Warehouse
    """
    print("🏗️ สร้าง Eating Behavior Warehouse...")

    # ดึงข้อมูลล่าสุด
    latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)

    # ฟังก์ชันแปลงคะแนน
    def convert_vegetable_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ไม่ทำเลย", 1: "1-3 วันต่อสัปดาห์", 2: "4-6 วันต่อสัปดาห์", 3: "ทุกวัน"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_dessert_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_sweet_drink_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_sweet_food_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "เติมทุกครั้ง", 1: "เติมบางครั้ง", 2: "ไม่เติมเลย"}
        return score_map.get(min(score, 2), score_map[2])

    def convert_fat_food_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_processed_food_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_instant_food_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_gourmet_powder_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ทุกวัน", 1: "4-6 วันต่อสัปดาห์", 2: "1-3 วันต่อสัปดาห์", 3: "ไม่ทำเลย"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_fish_sauce_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "เติมทุกครั้ง", 1: "เติมบางครั้ง", 2: "ไม่เติมเลย"}
        return score_map.get(min(score, 2), score_map[2])

    def handle_score_value(value):
        return 999 if pd.isna(value) or value == -1 else int(value)

    # สร้าง DataFrame
    eating_behavior_warehouse = pd.DataFrame({
        'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
        'member_id': latest_habits['memberid'],
        'กินผัก_score': latest_habits['vegetable'].apply(handle_score_value),
        'กินผัก': latest_habits['vegetable'].apply(convert_vegetable_score),
        'กินขนมหวาน_score': latest_habits['dessert'].apply(handle_score_value),
        'กินขนมหวาน': latest_habits['dessert'].apply(convert_dessert_score),
        'ดื่มเครื่องดื่มรสหวาน_score': latest_habits['sweet_drink'].apply(handle_score_value),
        'ดื่มเครื่องดื่มรสหวาน': latest_habits['sweet_drink'].apply(convert_sweet_drink_score),
        'เติมน้ำตาลน้ำเชื่อม_score': latest_habits['sweet_food'].apply(handle_score_value),
        'เติมน้ำตาลน้ำเชื่อม': latest_habits['sweet_food'].apply(convert_sweet_food_score),
        'อาหารไขมันสูง_score': latest_habits['fat_food'].apply(handle_score_value),
        'อาหารไขมันสูง': latest_habits['fat_food'].apply(convert_fat_food_score),
        'กินอาหารแปรรูป_score': latest_habits['processed_food'].apply(handle_score_value),
        'กินอาหารแปรรูป': latest_habits['processed_food'].apply(convert_processed_food_score),
        'กินอาหารสำเร็จรูป_score': latest_habits['instant_food'].apply(handle_score_value),
        'กินอาหารสำเร็จรูป': latest_habits['instant_food'].apply(convert_instant_food_score),
        'กินอาหารปรุงรสด้วยผงปรุงรส_score': latest_habits['gourmet_powder'].apply(handle_score_value),
        'กินอาหารปรุงรสด้วยผงปรุงรส': latest_habits['gourmet_powder'].apply(convert_gourmet_powder_score),
        'เติมน้ำปลา_score': latest_habits['fish_sauce'].apply(handle_score_value),
        'เติมน้ำปลา': latest_habits['fish_sauce'].apply(convert_fish_sauce_score)
    })

    print(f"✅ สร้าง Eating Behavior Warehouse: {len(eating_behavior_warehouse)} แถว")
    return eating_behavior_warehouse

# สร้าง Warehouse
eating_behavior_wh = create_eating_behavior_warehouse(ncds_habit)

# แสดงผลลัพธ์
print("\n📋 ตัวอย่างข้อมูล Eating Behavior Warehouse:")
eating_behavior_wh.head(10)



🏗️ สร้าง Eating Behavior Warehouse...
✅ สร้าง Eating Behavior Warehouse: 100 แถว

📋 ตัวอย่างข้อมูล Eating Behavior Warehouse:


,date,member_id,กินผัก_score,กินผัก,กินขนมหวาน_score,กินขนมหวาน,ดื่มเครื่องดื่มรสหวาน_score,ดื่มเครื่องดื่มรสหวาน,เติมน้ำตาลน้ำเชื่อม_score,เติมน้ำตาลน้ำเชื่อม,อาหารไขมันสูง_score,อาหารไขมันสูง,กินอาหารแปรรูป_score,กินอาหารแปรรูป,กินอาหารสำเร็จรูป_score,กินอาหารสำเร็จรูป,กินอาหารปรุงรสด้วยผงปรุงรส_score,กินอาหารปรุงรสด้วยผงปรุงรส,เติมน้ำปลา_score,เติมน้ำปลา
76,2025-06-02,10000036,3,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,1,เติมบางครั้ง,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,0,ทุกวัน,1,4-6 วันต่อสัปดาห์,2,ไม่เติมเลย
113,2025-06-10,10000058,999,999,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,0,เติมทุกครั้ง,0,ทุกวัน,0,ทุกวัน,3,ไม่ทำเลย,3,ไม่ทำเลย,0,เติมทุกครั้ง
53,2025-06-12,10000025,1,1-3 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,1,เติมบางครั้ง,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,2,ไม่เติมเลย
81,2025-06-12,10000040,2,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,999,999,2,ไม่เติมเลย,1,4-6 วันต่อสัปดาห์,3,ไม่ทำเลย,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,2,ไม่เติมเลย
141,2025-06-13,10000076,0,ไม่ทำเลย,999,999,999,999,1,เติมบางครั้ง,2,1-3 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,3,ไม่ทำเลย,1,เติมบางครั้ง
79,2025-06-17,10000038,999,999,2,1-3 วันต่อสัปดาห์,0,ทุกวัน,2,ไม่เติมเลย,3,ไม่ทำเลย,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,0,เติมทุกครั้ง
127,2025-06-23,10000066,1,1-3 วันต่อสัปดาห์,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,2,ไม่เติมเลย,1,4-6 วันต่อสัปดาห์,999,999,0,ทุกวัน,2,1-3 วันต่อสัปดาห์,2,ไม่เติมเลย
111,2025-06-30,10000056,3,ทุกวัน,3,ไม่ทำเลย,2,1-3 วันต่อสัปดาห์,1,เติมบางครั้ง,1,4-6 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,1,4-6 วันต่อสัปดาห์,0,ทุกวัน,0,เติมทุกครั้ง
80,2025-07-03,10000039,3,ทุกวัน,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,1,เติมบางครั้ง,2,1-3 วันต่อสัปดาห์,2,1-3 วันต่อสัปดาห์,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,1,เติมบางครั้ง
43,2025-07-04,10000019,2,4-6 วันต่อสัปดาห์,999,999,3,ไม่ทำเลย,0,เติมทุกครั้ง,3,ไม่ทำเลย,1,4-6 วันต่อสัปดาห์,0,ทุกวัน,0,ทุกวัน,0,เติมทุกครั้ง


### Exercise 3.4: Comprehensive Behavior Warehouse

**โจทย์**: สร้างตารางพฤติกรรมรวม

**ข้อมูลที่ต้องแปลง**:
- การเคลื่อนไหว (movement)
- การสูบบุหรี่ (smoking)
- การนอนหลับ (sleep)
- ภาวะซึมเศร้า (2q_1)
- ความเครียด (2q_2)
- การดื่มแอลกอฮอล์ (alcohol)
- การออกกำลังกาย (exercise)

In [26]:
def create_comprehensive_behavior_warehouse(ncds_habit):
    """
    สร้างตาราง Comprehensive Behavior Warehouse

    Returns:
        DataFrame: Comprehensive Behavior Warehouse
    """
    print("🏗️ สร้าง Comprehensive Behavior Warehouse...")

    # ดึงข้อมูลล่าสุด
    latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)

    # ฟังก์ชันแปลงค่า
    def convert_movement_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ไม่ทำเลย", 1: "1-2 วันต่อสัปดาห์", 2: "มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์"}
        return score_map.get(min(score, 2), score_map[2])

    def convert_smoking_code(code):
        if pd.isna(code):
            return 999
        code = int(code)
        code_map = {1: "สูบ", -1: "ไม่สูบ"}
        return code_map.get(code, 999)

    def convert_sleep_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "0 - 2 วันต่อสัปดาห์", 1: "3 - 5 วันต่อสัปดาห์", 2: "6 - 7 วันต่อสัปดาห์"}
        return score_map.get(min(score, 2), score_map[2])

    def convert_depression_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ไม่มีความเสี่ยง", 1: "มีความเสี่ยง"}
        return score_map.get(min(score, 1), score_map[1])

    def convert_stress_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "แทบไม่มี", 1: "เป็นบางครั้ง", 2: "บ่อยครั้ง", 3: "เป็นประจำ"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_alcohol_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {0: "ไม่ดื่มเลย", 1: "ดื่ม 1 - 3 วันต่อสัปดาห์", 2: "ดื่ม 4 - 6 วันต่อสัปดาห์", 3: "ดื่มเป็นประจำทุกวัน"}
        return score_map.get(min(score, 3), score_map[3])

    def convert_exercise_score(score):
        if pd.isna(score) or score == -1:
            return 999
        score = int(score)
        score_map = {
            0: "ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก",
            1: "ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์",
            2: "ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์",
            3: "ออกกำลังกายหนัก เล่นกีฬา 6-7 วัน/สัปดาห์",
            4: "ออกกำลังกายหนักมากเป็นนักกีฬา"
        }
        return score_map.get(min(score, 4), score_map[4])

    def handle_behavior_score_value(value):
        return 999 if pd.isna(value) or value == -1 else int(value)

    def handle_smoking_code_value(value):
        return 999 if pd.isna(value) else int(value)

    # สร้าง DataFrame
    comprehensive_behavior_warehouse = pd.DataFrame({
        'date': pd.to_datetime(latest_habits['dateadd']).dt.date,
        'member_id': latest_habits['memberid'],
        'การเคลื่อนไหวร่างกาย_score': latest_habits['movement'].apply(handle_behavior_score_value),
        'การเคลื่อนไหวร่างกาย': latest_habits['movement'].apply(convert_movement_score),
        'สูบบุหรี่_code': latest_habits['smoking'].apply(handle_smoking_code_value),
        'สูบบุหรี่': latest_habits['smoking'].apply(convert_smoking_code),
        'การนอนหลับ_score': latest_habits['sleep'].apply(handle_behavior_score_value),
        'การนอนหลับ': latest_habits['sleep'].apply(convert_sleep_score),
        'ภาวะซึมเศร้า_score': latest_habits['2q_1'].apply(handle_behavior_score_value),
        'ภาวะซึมเศร้า': latest_habits['2q_1'].apply(convert_depression_score),
        'ความเครียด_score': latest_habits['2q_2'].apply(handle_behavior_score_value),
        'ความเครียด': latest_habits['2q_2'].apply(convert_stress_score),
        'การดื่มแอล_score': latest_habits['alcohol'].apply(handle_behavior_score_value),
        'การดื่มแอล': latest_habits['alcohol'].apply(convert_alcohol_score),
        'ออกกำลังกาย_score': latest_habits['exercise'].apply(handle_behavior_score_value),
        'ออกกำลังกาย': latest_habits['exercise'].apply(convert_exercise_score)
    })

    print(f"✅ สร้าง Comprehensive Behavior Warehouse: {len(comprehensive_behavior_warehouse)} แถว")
    return comprehensive_behavior_warehouse

# สร้าง Warehouse
comprehensive_behavior_wh = create_comprehensive_behavior_warehouse(ncds_habit)

# แสดงผลลัพธ์
print("\n📋 ตัวอย่างข้อมูล Comprehensive Behavior Warehouse:")
comprehensive_behavior_wh.head(10)


🏗️ สร้าง Comprehensive Behavior Warehouse...
✅ สร้าง Comprehensive Behavior Warehouse: 100 แถว

📋 ตัวอย่างข้อมูล Comprehensive Behavior Warehouse:


,date,member_id,การเคลื่อนไหวร่างกาย_score,การเคลื่อนไหวร่างกาย,สูบบุหรี่_code,สูบบุหรี่,การนอนหลับ_score,การนอนหลับ,ภาวะซึมเศร้า_score,ภาวะซึมเศร้า,ความเครียด_score,ความเครียด,การดื่มแอล_score,การดื่มแอล,ออกกำลังกาย_score,ออกกำลังกาย
76,2025-06-02,10000036,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,-1,ไม่สูบ,999,999,0,ไม่มีความเสี่ยง,0,แทบไม่มี,3,ดื่มเป็นประจำทุกวัน,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก
113,2025-06-10,10000058,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,1,เป็นบางครั้ง,3,ดื่มเป็นประจำทุกวัน,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
53,2025-06-12,10000025,1,1-2 วันต่อสัปดาห์,-1,ไม่สูบ,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,999,999,0,ไม่ดื่มเลย,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
81,2025-06-12,10000040,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,1,3 - 5 วันต่อสัปดาห์,0,ไม่มีความเสี่ยง,0,แทบไม่มี,1,ดื่ม 1 - 3 วันต่อสัปดาห์,2,ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์
141,2025-06-13,10000076,1,1-2 วันต่อสัปดาห์,0,999,2,6 - 7 วันต่อสัปดาห์,1,มีความเสี่ยง,0,แทบไม่มี,2,ดื่ม 4 - 6 วันต่อสัปดาห์,0,ไม่ออกกำลังกายหรือออกกำลังกายน้อยมาก
79,2025-06-17,10000038,2,มากกว่าหรือเท่ากับ 3 วันต่อสัปดาห์,0,999,999,999,1,มีความเสี่ยง,999,999,2,ดื่ม 4 - 6 วันต่อสัปดาห์,999,999
127,2025-06-23,10000066,1,1-2 วันต่อสัปดาห์,-1,ไม่สูบ,2,6 - 7 วันต่อสัปดาห์,1,มีความเสี่ยง,0,แทบไม่มี,1,ดื่ม 1 - 3 วันต่อสัปดาห์,4,ออกกำลังกายหนักมากเป็นนักกีฬา
111,2025-06-30,10000056,0,ไม่ทำเลย,0,999,999,999,0,ไม่มีความเสี่ยง,0,แทบไม่มี,999,999,2,ออกกำลังกายปานกลาง เล่นกีฬา 3-5 วัน/สัปดาห์
80,2025-07-03,10000039,0,ไม่ทำเลย,0,999,1,3 - 5 วันต่อสัปดาห์,1,มีความเสี่ยง,1,เป็นบางครั้ง,1,ดื่ม 1 - 3 วันต่อสัปดาห์,1,ออกกำลังกายน้อย เล่นกีฬา 1-3 วัน/สัปดาห์
43,2025-07-04,10000019,0,ไม่ทำเลย,-1,ไม่สูบ,1,3 - 5 วันต่อสัปดาห์,0,ไม่มีความเสี่ยง,0,แทบไม่มี,2,ดื่ม 4 - 6 วันต่อสัปดาห์,3,ออกกำลังกายหนัก เล่นกีฬา 6-7 วัน/สัปดาห์


In [ ]:
# TODO: สร้าง Disease Risk Warehouse
# Hint: 1. ดึงค่าจาก memberdata (sys/dias, bmi, waistline, height)
#       2. สร้างฟังก์ชันคำนวณความเสี่ยงทั้ง 3 ประเภท
#       3. Loop แต่ละสมาชิกและคำนวณ
#       4. สร้าง DataFrame จากผลลัพธ์

# เขียนโค้ดที่นี่



### Exercise 3.5: Disease Risk Warehouse

**โจทย์**: สร้างตารางความเสี่ยงโรค 3 ประเภท

**ความเสี่ยงที่ต้องคำนวณ**:
1. **โรคความดันโลหิตสูง**: จากค่า sys/dias
2. **เสี่ยงโรคเบาหวาน**: ใช้ FINDRISC algorithm (คำนวณจาก age, gender, BMI, waistline, sys/dias, relatives_diabetes)
3. **เสี่ยงโรคหัวใจและหลอดเลือด**: คำนวณจาก age, gender, smoking, diabetes, sys, waist/height ratio

**โครงสร้างข้อมูล**:
- แต่ละสมาชิกจะมี 3 แถว (1 แถวต่อ 1 ประเภทความเสี่ยง)
- คอลัมน์: member_id, ภาวะเสี่ยง, ระดับความเสี่ยง, คะแนน

**Hint**: ดูตัวอย่างจาก detailed_health_warehouse_tutorial.ipynb

In [28]:
#Exercise 3.5: Disease Risk Warehouse

def create_disease_risk_warehouse(ncds_profile, ncds_record, ncds_habit, memberdata):
    """
    สร้างตาราง Disease Risk Warehouse


    คำนวณความเสี่ยง 3 ประเภท:
    1. โรคความดันโลหิตสูง (Hypertension)
    2. เสี่ยงโรคเบาหวาน (Diabetes Risk - FINDRISC)
    3. เสี่ยงโรคหัวใจและหลอดเลือด (Cardiovascular Risk)


    Returns:
        DataFrame: Disease Risk Warehouse
    """
    print("🏗️ สร้าง Disease Risk Warehouse...")


    # ดึงข้อมูลล่าสุด
    latest_profiles = ncds_profile.sort_values('dateadd').groupby('memberid').tail(1)
    latest_records = ncds_record.sort_values('dateadd').groupby('memberid').tail(1)
    latest_habits = ncds_habit.sort_values('dateadd').groupby('memberid').tail(1)
    latest_memberdata = memberdata.sort_values(['memberid', 'name', 'dateadd']).groupby(['memberid', 'name']).tail(1)


    # ฟังก์ชันดึงค่าจาก memberdata
    def get_memberdata_value(member_id, name):
        try:
            result = latest_memberdata[(latest_memberdata['memberid'] == member_id) &
                                      (latest_memberdata['name'] == name)]['value']
            if len(result) > 0:
                value = result.iloc[0]
                if pd.notna(value) and str(value) not in ['nan', '- - -', '', 'null']:
                    if name == 'sys/dias':
                        return str(value)
                    else:
                        try:
                            return float(value)
                        except:
                            return str(value)
            return None
        except:
            return None


    # ฟังก์ชันแยกความดัน
    def parse_blood_pressure(bp_value):
        if pd.isna(bp_value) or str(bp_value) in ['nan', 'None', '- - -', '']:
            return None, None
        try:
            bp_str = str(bp_value).strip()
            if '/' in bp_str:
                parts = bp_str.split('/')
                if len(parts) == 2:
                    sys_val = float(parts[0].strip()) if parts[0].strip().replace('.', '').isdigit() else None
                    dias_val = float(parts[1].strip()) if parts[1].strip().replace('.', '').isdigit() else None
                    return sys_val, dias_val
        except:
            pass
        return None, None


    # ฟังก์ชันคำนวณระดับความดัน
    def calculate_hypertension_status(sys, dias):
        if pd.isna(sys) or pd.isna(dias):
            return None


        if sys > 50 and sys < 130:
            sys_level = "ปกติ"
        elif sys >= 130 and sys < 140:
            sys_level = "สูง"
        elif sys >= 140 and sys < 180:
            sys_level = "ค่อนข้างสูง"
        elif sys >= 180:
            sys_level = "สูงมาก"
        else:
            sys_level = None


        if dias > 50 and dias < 85:
            dias_level = "ปกติ"
        elif dias >= 85 and dias < 90:
            dias_level = "สูง"
        elif dias >= 90 and dias < 110:
            dias_level = "ค่อนข้างสูง"
        elif dias >= 110:
            dias_level = "สูงมาก"
        else:
            dias_level = None


        if sys_level == "สูงมาก" or dias_level == "สูงมาก":
            return "สูงมาก"
        elif sys_level == "ค่อนข้างสูง" or dias_level == "ค่อนข้างสูง":
            return "ค่อนข้างสูง"
        elif sys_level == "สูง" or dias_level == "สูง":
            return "สูง"
        elif sys_level == "ปกติ" or dias_level == "ปกติ":
            return "ปกติ"
        return None


    # ฟังก์ชันคำนวณความเสี่ยงเบาหวาน (FINDRISC)
    def calculate_diabetes_risk_score(age, gender, bmi, waistline, sys, dias, relatives_diabetes):
        if pd.isna(age) or pd.isna(gender):
            return None, None, "ข้อมูลไม่ครบถ้วน"


        total_score = 0
        probability_sum = 0


        # Age score
        if age < 45:
            age_score, age_prob = 1, -0.0702134
        elif 45 <= age < 60:
            age_score, age_prob = 1, 0.2718858
        else:
            age_score, age_prob = 1, 0.6043599


        total_score += age_score
        probability_sum += age_score * age_prob


        # Gender score
        if gender == 0:  # หญิง
            gender_score, gender_prob = 0, 0
        else:  # ชาย
            gender_score, gender_prob = 2, 0.4422573


        total_score += gender_score
        probability_sum += gender_score * gender_prob


        # BMI score
        if not pd.isna(bmi):
            if bmi < 23:
                bmi_score, bmi_prob = 0, 0
            elif 23 <= bmi < 27.5:
                bmi_score, bmi_prob = 1, 0.6958621
            else:
                bmi_score, bmi_prob = 1, 1.235097


            total_score += bmi_score
            probability_sum += bmi_score * bmi_prob


        # Waistline score
        if not pd.isna(waistline):
            if (gender == 1 and waistline < 90) or (gender == 0 and waistline < 80):
                waist_score, waist_prob = 0, 0
            else:
                waist_score, waist_prob = 1, 0.5567118


            total_score += waist_score
            probability_sum += waist_score * waist_prob


        # Blood pressure score
        if not pd.isna(sys) and not pd.isna(dias):
            if sys < 140 and dias < 90:
                bp_score, bp_prob = 0, 0
            else:
                bp_score, bp_prob = 1, 0.6409517


            total_score += bp_score
            probability_sum += bp_score * bp_prob


        # Family history score
        if not pd.isna(relatives_diabetes) and relatives_diabetes == 1:
            family_score, family_prob = 1, 1.081356
        else:
            family_score, family_prob = 0, 0


        total_score += family_score
        probability_sum += family_score * family_prob


        # Calculate probability
        constant = -3.580397
        sum_with_constant = probability_sum + constant
        exp_value = np.exp(sum_with_constant)
        risk_probability = exp_value / (1 + exp_value)
        risk_percentage = risk_probability * 100


        return total_score, risk_percentage, None


    # ฟังก์ชันระดับความเสี่ยงเบาหวาน
    def get_diabetes_risk_level(score):
        if pd.isna(score):
            return None
        if score <= 2:
            return "ปกติ"
        elif 3 <= score <= 5:
            return "สูง"
        elif 6 <= score <= 8:
            return "ค่อนข้างสูง"
        else:
            return "สูงมาก"


    # ฟังก์ชันคำนวณความเสี่ยงหัวใจและหลอดเลือด
    def calculate_cardiovascular_risk(age, gender, smoking, dm, sbp, waist, height):
        if any(pd.isna(val) for val in [age, gender, sbp, waist, height]):
            return None, None


        sex = 1 if gender == 1 else 0
        smoke_val = 1 if smoking == 1 else 0
        dm_val = 1 if dm == 1 else 0


        full_score = (0.079 * age) + (0.128 * sex) + (0.019350987 * sbp) + (0.58454 * dm_val) + (3.512566 * (waist / height)) + (0.459 * smoke_val)
        p_full_score = (1 - (0.978296 ** np.exp(full_score - 7.720484))) * 100


        return full_score, p_full_score


    # ฟังก์ชันระดับความเสี่ยงหัวใจ
    def get_cardiovascular_risk_level(percentage):
        if pd.isna(percentage):
            return None
        if percentage < 10:
            return "ปกติ"
        elif 10 <= percentage < 20:
            return "สูง"
        elif 20 <= percentage < 30:
            return "ค่อนข้างสูง"
        else:
            return "สูงมาก"


    # ดึง member IDs ทั้งหมด
    all_member_ids = latest_memberdata['memberid'].unique()
    print(f"📊 จำนวน member_id ทั้งหมด: {len(all_member_ids)}")


    disease_risk_records = []


    for member_id in all_member_ids:
        # ดึงข้อมูลของสมาชิก
        profile_data = latest_profiles[latest_profiles['memberid'] == member_id]
        age = profile_data['age'].iloc[0] if len(profile_data) > 0 else None
        gender = profile_data['gender'].iloc[0] if len(profile_data) > 0 else None


        record_data = latest_records[latest_records['memberid'] == member_id]
        relatives_diabetes = record_data['relatives_diabetes'].iloc[0] if len(record_data) > 0 else None
        record_diabetes = record_data['record_diabetes'].iloc[0] if len(record_data) > 0 else None


        habit_data = latest_habits[latest_habits['memberid'] == member_id]
        smoking = habit_data['smoking'].iloc[0] if len(habit_data) > 0 else None


        # ดึงข้อมูลการวัด
        sys_dias = get_memberdata_value(member_id, 'sys/dias')
        bmi = get_memberdata_value(member_id, 'bmi')
        waistline = get_memberdata_value(member_id, 'waistline')
        height = get_memberdata_value(member_id, 'height')


        sys, dias = parse_blood_pressure(sys_dias)


        # 1. ความดันโลหิตสูง
        hypertension_status = calculate_hypertension_status(sys, dias)
        hypertension_score = f"{int(sys) if sys else ''}/{int(dias) if dias else ''}" if sys or dias else 999


        disease_risk_records.append({
            'member_id': member_id,
            'ภาวะเสี่ยง': 'โรคความดันโลหิตสูง',
            'ระดับความเสี่ยง': hypertension_status if hypertension_status is not None else "ไม่ระบุ",
            'คะแนน': hypertension_score
        })


        # 2. เสี่ยงโรคเบาหวาน
        diabetes_risk_score, diabetes_risk_percentage, error = calculate_diabetes_risk_score(
            age, gender, bmi, waistline, sys, dias, relatives_diabetes
        )


        diabetes_risk_level = get_diabetes_risk_level(diabetes_risk_score) if diabetes_risk_score is not None else "ไม่ระบุ"
        diabetes_score_text = f"{diabetes_risk_score}/{diabetes_risk_percentage:.2f}" if diabetes_risk_score is not None and diabetes_risk_percentage is not None else 999


        disease_risk_records.append({
            'member_id': member_id,
            'ภาวะเสี่ยง': 'เสี่ยงโรคเบาหวาน',
            'ระดับความเสี่ยง': diabetes_risk_level,
            'คะแนน': diabetes_score_text
        })


        # 3. เสี่ยงโรคหัวใจและหลอดเลือด
        cardio_full_score, cardio_percentage = None, None
        cardio_risk_level = None
        cardio_score_text = 999


        if all(val is not None for val in [age, gender, sys, waistline, height]):
            cardio_full_score, cardio_percentage = calculate_cardiovascular_risk(
                age, gender, smoking, record_diabetes, sys, waistline, height
            )


            if cardio_percentage is not None:
                cardio_risk_level = get_cardiovascular_risk_level(cardio_percentage)
                cardio_score_text = f"{cardio_percentage:.2f}%"


        disease_risk_records.append({
            'member_id': member_id,
            'ภาวะเสี่ยง': 'เสี่ยงโรคหัวใจและหลอดเลือด',
            'ระดับความเสี่ยง': cardio_risk_level if cardio_risk_level is not None else "ไม่ระบุ",
            'คะแนน': cardio_score_text
        })


    disease_risk_warehouse = pd.DataFrame(disease_risk_records)
    print(f"✅ สร้าง Disease Risk Warehouse: {len(disease_risk_warehouse)} แถว")
    return disease_risk_warehouse


# สร้าง Warehouse
disease_risk_wh = create_disease_risk_warehouse(ncds_profile, ncds_record, ncds_habit, memberdata)


# แสดงผลลัพธ์
print("\n📋 ตัวอย่างข้อมูล Disease Risk Warehouse:")
disease_risk_wh.head(15)


🏗️ สร้าง Disease Risk Warehouse...
📊 จำนวน member_id ทั้งหมด: 100
✅ สร้าง Disease Risk Warehouse: 300 แถว

📋 ตัวอย่างข้อมูล Disease Risk Warehouse:


,member_id,ภาวะเสี่ยง,ระดับความเสี่ยง,คะแนน
0,10000000,โรคความดันโลหิตสูง,ปกติ,120/71
1,10000000,เสี่ยงโรคเบาหวาน,สูง,4/23.35
2,10000000,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,5.27%
3,10000001,โรคความดันโลหิตสูง,ค่อนข้างสูง,144/78
4,10000001,เสี่ยงโรคเบาหวาน,สูง,4/19.54
5,10000001,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,7.07%
6,10000002,โรคความดันโลหิตสูง,ค่อนข้างสูง,116/93
7,10000002,เสี่ยงโรคเบาหวาน,สูง,4/29.41
8,10000002,เสี่ยงโรคหัวใจและหลอดเลือด,ปกติ,5.97%
9,10000003,โรคความดันโลหิตสูง,ค่อนข้างสูง,124/100


---

## 📤 Part 4: อัปโหลดไปยัง Google Sheets (15 นาที)

### Exercise 4.1: Authentication

**โจทย์**: Authenticate กับ Google เพื่อใช้งาน Google Sheets API

In [30]:
# TODO: Authenticate กับ Google
# Hint: ใช้ from google.colab import auth
#       auth.authenticate_user()
#       แล้ว authorize กับ gspread

# เขียนโค้ดที่นี่
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default

# Authenticate gspread
creds, _ = default()
gc = gspread.authorize(creds)

print("✅ Authentication with Google Sheets successful")

✅ Authentication with Google Sheets successful


In [35]:
# สร้าง Google Spreadsheet ใหม่
spreadsheet_name = f"Health Data Warehouse - {datetime.datetime.now().strftime('%Y%m%d_%H%M%S')}"
sh = gc.create(spreadsheet_name)


print(f"✅ สร้าง Spreadsheet: {spreadsheet_name}")
print(f"🔗 URL: https://docs.google.com/spreadsheets/d/{sh.id}")


# แชร์ให้ทุกคนดูได้ (optional)
sh.share('', perm_type='anyone', role='reader')
print("✅ แชร์เป็น Public (ดูได้อย่างเดียว)")

✅ สร้าง Spreadsheet: Health Data Warehouse - 20251006_045737
🔗 URL: https://docs.google.com/spreadsheets/d/1htMDJYhiy4zeNHvs7HANPp_NxwIho3Ew56yR61S5Atc
✅ แชร์เป็น Public (ดูได้อย่างเดียว)


### Exercise 4.3: อัปโหลดข้อมูล

**โจทย์**: อัปโหลดข้อมูลทั้ง 5 warehouses ไปยัง Google Sheets โดยแต่ละ warehouse เป็น 1 sheet

**ชื่อ Sheet**:
- 01-ข้อมูลผู้ตรวจสุขภาพ
- 02-ประวัติโรคNCDs
- 03-พฤติกรรมการกิน
- 04-พฤติกรรมรวม
- 05-ความเสี่ยงโรค

In [36]:
def upload_to_sheet(spreadsheet, df, sheet_name):
    """
    อัปโหลด DataFrame ไปยัง Google Sheet


    Args:
        spreadsheet: Google Spreadsheet object
        df: DataFrame ที่จะอัปโหลด
        sheet_name: ชื่อ sheet
    """
    try:
        # ลองหา worksheet ที่มีอยู่
        worksheet = spreadsheet.worksheet(sheet_name)
    except:
        # ถ้าไม่มีให้สร้างใหม่
        worksheet = spreadsheet.add_worksheet(title=sheet_name, rows=len(df)+10, cols=len(df.columns)+5)


    # ล้างข้อมูลเดิม
    worksheet.clear()


    # แปลง DataFrame เป็น list
    df_clean = df.copy()
    df_clean = df_clean.replace([float('inf'), float('-inf')], None)
    df_clean = df_clean.fillna('')


    # แปลง date และ datetime เป็น string
    for col in df_clean.columns:
        if df_clean[col].dtype == 'object':
            df_clean[col] = df_clean[col].apply(lambda x: str(x) if x != '' else '')


    # อัปโหลด
    data_to_upload = [df_clean.columns.values.tolist()] + df_clean.values.tolist()
    worksheet.update(values=data_to_upload, range_name='A1')


    print(f"✅ อัปโหลด {sheet_name}: {len(df)} แถว")


# อัปโหลดทั้ง 5 warehouses
print("\n📤 กำลังอัปโหลดข้อมูลไปยัง Google Sheets...\n")


upload_to_sheet(sh, health_profile_wh, '01-ข้อมูลผู้ตรวจสุขภาพ')
upload_to_sheet(sh, ncds_disease_wh, '02-ประวัติโรคNCDs')
upload_to_sheet(sh, eating_behavior_wh, '03-พฤติกรรมการกิน')
upload_to_sheet(sh, comprehensive_behavior_wh, '04-พฤติกรรมรวม')
upload_to_sheet(sh, disease_risk_wh, '05-ความเสี่ยงโรค')


# ลบ Sheet1 ที่สร้างมาตั้งแต่แรก
try:
    worksheet = sh.worksheet('Sheet1')
    sh.del_worksheet(worksheet)
except:
    pass


print("\n🎉 อัปโหลดข้อมูลทั้งหมดสำเร็จ!")
print(f"\n🔗 เปิดดูได้ที่: https://docs.google.com/spreadsheets/d/{sh.id}")




📤 กำลังอัปโหลดข้อมูลไปยัง Google Sheets...

✅ อัปโหลด 01-ข้อมูลผู้ตรวจสุขภาพ: 100 แถว
✅ อัปโหลด 02-ประวัติโรคNCDs: 308 แถว
✅ อัปโหลด 03-พฤติกรรมการกิน: 100 แถว
✅ อัปโหลด 04-พฤติกรรมรวม: 100 แถว
✅ อัปโหลด 05-ความเสี่ยงโรค: 300 แถว

🎉 อัปโหลดข้อมูลทั้งหมดสำเร็จ!

🔗 เปิดดูได้ที่: https://docs.google.com/spreadsheets/d/1htMDJYhiy4zeNHvs7HANPp_NxwIho3Ew56yR61S5Atc


---

## 📝 Part 5: สรุปและคำถาม (10 นาที)

### คำถามท้ายบท:

**Q1**: ทำไมต้องดึงข้อมูล "ล่าสุด" ของแต่ละสมาชิก ไม่ใช่ข้อมูลทั้งหมด?

**คำตอบ**: _______________________________________________________

---

**Q2**: ในตาราง NCDs Disease ทำไมถึงมีหลายแถวต่อ 1 สมาชิก?

**คำตอบ**: _______________________________________________________

---

**Q3**: อะไรคือความแตกต่างระหว่าง "_score" และ "label" ในตาราง Eating Behavior?

**คำตอบ**: _______________________________________________________

---

**Q4**: ลองคิดว่าถ้าจะเพิ่ม Warehouse ตารางที่ 6 ควรจะเป็นตารางอะไร? และมีข้อมูลอะไรบ้าง?

**คำตอบ**: _______________________________________________________

---

## ✅ Checklist สำหรับการส่งงาน

- [ ] สร้าง Health Profile Warehouse ได้อย่างน้อย 100 แถว
- [ ] สร้าง NCDs Disease Warehouse ได้
- [ ] สร้าง Eating Behavior Warehouse ได้
- [ ] สร้าง Comprehensive Behavior Warehouse ได้
- [ ] สร้าง Disease Risk Warehouse ได้
- [ ] อัปโหลดข้อมูลไปยัง Google Sheets สำเร็จ
- [ ] ตอบคำถามท้ายบทครบทั้ง 4 ข้อ
- [ ] แชร์ Google Sheets ให้อาจารย์ดูได้

## 📤 วิธีการส่งงาน

1. Save notebook นี้ (Download .ipynb)
2. Copy URL ของ Google Sheets ที่สร้าง
3. ส่งทั้ง 2 อย่างใน Google Classroom

---

**Google Sheets URL**: https://docs.google.com/spreadsheets/d/1htMDJYhiy4zeNHvs7HANPp_NxwIho3Ew56yR61S5Atc/edit?gid=314898792#gid=314898792

**หมายเหตุ**: กรุณาแชร์ Google Sheets ให้อาจารย์สามารถดูได้ (Viewer)

---

## 🎓 เกณฑ์การให้คะแนน (100 คะแนน)

| หัวข้อ | คะแนน |
|--------|-------|
| Part 1-2: เตรียมสภาพแวดล้อมและดึงข้อมูล | 15 |
| Part 3.1: Health Profile Warehouse | 20 |
| Part 3.2: NCDs Disease Warehouse | 12 |
| Part 3.3: Eating Behavior Warehouse | 12 |
| Part 3.4: Comprehensive Behavior Warehouse | 12 |
| Part 3.5: Disease Risk Warehouse | 19 |
| Part 4: อัปโหลดไปยัง Google Sheets | 5 |
| Part 5: คำถามท้ายบท | 5 |
| **รวม** | **100** |

---

**สิ่งที่ได้เรียนรู้**:
✅ การทำ Data Transformation  
✅ การใช้ Pandas สำหรับ Data Wrangling  
✅ การสร้าง Data Warehouse  
✅ การคำนวณความเสี่ยงโรค (FINDRISC, Cardiovascular Risk)  
✅ การใช้ Google Sheets API  

**Good Luck! 🎓**

In [ ]:
# ปิดการเชื่อมต่อ Database
conn.close()
print("✅ ปิดการเชื่อมต่อ Database เรียบร้อยแล้ว")